# Numerical Benchmarks

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from lindblad_channels import *
from quantum_tools import *
from experimental import *
from spam import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Recover SPAM and Map

### Generate True Model, Full POVM and Inital

In [4]:
def generate_spam_benchmark(n=3, c1=1, c2=1):
    d = 2**n

    init_target = InitialState(d, c=c1)
    povm_target = POVM(d, c=c2)

    spam_target = SPAM(init = init_target,
                       povm = povm_target)
    
    return spam_target


def generate_spam_data(spam_target, N_spam=None, noise=0):
    n = int(np.log2(spam_target.d))
    inputs_spam, _ = generate_pauliInput_circuits(n)
    N_spam = inputs_spam.shape[0]

    state = tf.repeat(spam_target.init.init[None,:,:], N_spam, axis=0)
    targets_spam = measurement(state, U_basis = inputs_spam, povm = spam_target.povm.povm)

    #add noise
    targets_spam = add_noise_to_probs(targets_spam, noise=noise)
    return inputs_spam, targets_spam


def generate_map_data(channel_target, spam_target, N_map=None, noise=0):
    n = channel_target.n
    inputs_map, _ = generate_pauli_circuits(n = n, 
                                            circuit_target=None,  
                                            trace=False,
                                            N=N_map)
    U_prep, U_basis = inputs_map

    N_map = U_prep.shape[0]
    state = tf.repeat(tf.expand_dims(spam_target.init.init, axis=0), N_map, axis=0)
    state = apply_unitary(state, U_prep)
    state = channel_target.apply_channel(state)
    targets_map = measurement(state, U_basis, spam_target.povm.povm)
    
    #add noise
    targets_map = add_noise_to_probs(targets_map, noise=noise)
    
    return inputs_map, targets_map


def model_pipeline(channel_target, noise, N_map = None, num_iter_spam=None, num_iter_map=None):
    # Make Benchmark
    #################################################################################
    n = 3
    d = 2**n
    
    spam_target = generate_spam_benchmark(n=n, c1=0.9, c2=0.9)
    inputs_spam, targets_spam = generate_spam_data(spam_target, N_spam=None, noise=noise)

    inputs_map, targets_map = generate_map_data(channel_target, spam_target, N_map=N_map, noise=noise)
    #################################################################################

    # Fit Models
    #################################################################################
    spam_model = SPAM(init = InitialState(d, c=0.9),
                      povm = CorruptionMatrix(d, c=0.9),
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    #spam_model.pretrain(300, verbose=False)

    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = num_iter_map,
                     verbose = False,
                    )

    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                               rank = d**2,
                                               spam = spam_model,
                                              ),
                            loss_function = ProbabilityMSE(),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            logger = Logger(loss_function = ProbabilityMSE(),
                                            loss_function_val = channel_fidelity_loss),
                           )

    model.train(inputs = inputs_map,
                targets = targets_map,
                inputs_val = None,
                targets_val = [channel_target],
                num_iter = num_iter_map,
                N = 500,
                )
    #################################################################################
    model.optimizer = None
    spam_model.optimizer = None
    model.inputs = None
    model.targets = None
    model.inputs_val = None
    model.targets_val = None

    return model

## Fit Models

In [5]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n

channel_target1 = CompactLindbladMap(d, 1, 1, 0.1)
model1 = model_pipeline(channel_target1, 
                        noise=1/np.sqrt(1024), 
                        N_map=2000-6**n, 
                        num_iter_spam = 2000,
                        num_iter_map = 2000)

channel_target2 = CompactLindbladMap(d, 16, 10000, 0.001)
model2 = model_pipeline(channel_target2, 
                        noise=1/np.sqrt(1024), 
                        N_map=2000-6**n, 
                        num_iter_spam = 2000,
                        num_iter_map = 2000)

channel_target3 = CompactLindbladMap(d, 16, 100, 0.001)
model3 = model_pipeline(channel_target3, 
                        noise=1/np.sqrt(1024), 
                        N_map=2000-6**n, 
                        num_iter_spam = 2000,
                        num_iter_map = 2000)

channel_target4 = CompactLindbladMap(d, 8, 1, 0.01)
model4 = model_pipeline(channel_target4, 
                        noise=1/np.sqrt(1024), 
                        N_map=2000-6**n, 
                        num_iter_spam = 2000,
                        num_iter_map = 2000)

saver([[channel_target1, channel_target2, channel_target3, channel_target4],
       [model1, model2, model3, model4]], data_path("atypical_maps_benchmark.model"))

  0%|          | 0/2000 [00:00<?, ?it/s]

0.0005817683328900032


  0%|          | 0/2000 [00:00<?, ?it/s]

0.2524419904157432 -0.08766725008272647
0.06171949678490715 -0.36323968798978745
0.016314720447863747 -0.6696420605484509
0.008517738903805184 -0.8029302246810687
0.006764104704229357 -0.85962091032573
0.0061559399901818124 -0.8890967388055686
0.00587405856080175 -0.9066648301793278
0.00572396714369966 -0.9174502768813971
0.005638702329338704 -0.9249953034922637
0.005576416972249685 -0.9299444291137022
0.005537916590660458 -0.9337667615000803
0.0055120909190161434 -0.9364369102749336
0.0054857766079517405 -0.9384180959513664
0.005471466711034589 -0.9401345723189476
0.0054703950615628445 -0.9412556923694003
0.005458818227422155 -0.9420303066122757
0.005449703417643162 -0.9428796169166822
0.005447692154006037 -0.943647881706282
0.005440068234720057 -0.9441755733854561
0.005441975803994386 -0.9445893647517913
0.005443045367879531 -0.9447073115788313


  0%|          | 0/2000 [00:00<?, ?it/s]

...

0.537977251208305 -0.16303926734635094
0.14914985913883333 -0.4173718131129478
0.015974069455456045 -0.7241724833398019
0.007980127349528068 -0.8047648245178135
0.006554386206893429 -0.83757463516688
0.005848235189344943 -0.8593282114458343
0.00548448329582082 -0.8742974888064742
0.005258565946530385 -0.8849649902217828
0.005122891108680794 -0.8923873459582711
0.0050342342080999716 -0.898056500193913
0.004964450568814712 -0.9023069530813025
0.004910394349423254 -0.9053990767176138
0.0048670789071556236 -0.9075203119237147
0.004845808343377309 -0.9092703997636111
0.00482205214686176 -0.9107832587217065
0.004802291845025632 -0.911670586181411
0.004790428842780477 -0.9120366144585846
0.00478685291314118 -0.9123762615046329
0.004772886729135632 -0.9125464005613652
0.0047664283554789254 -0.912593694956426
0.004770667615547937 -0.9127403817190828


  0%|          | 0/2000 [00:00<?, ?it/s]

0.0006263633972401861


  0%|          | 0/2000 [00:00<?, ?it/s]

0.12370942614921444 -0.4112373402658576
0.019292708543692704 -0.6316199647502514
0.007987219708114376 -0.7762639037022258
0.005933562570460459 -0.8344728385967343
0.005327596715541875 -0.8609362285268881
0.005053971447271981 -0.8741462212051431
0.004900273379458376 -0.8813767703840305
0.004800670433566611 -0.8851831304745916
0.0047393653505126395 -0.8862915342707123
0.004699620685564887 -0.886126739958099
0.004661172618186593 -0.8852737364668173
0.00463985346670873 -0.8840948188366017
0.004623585983964953 -0.8824278628139108
0.004614382096549913 -0.8804622640634665
0.004610193056081025 -0.8787534342780619
0.00460298481708815 -0.8772013971589401
0.004599198977733526 -0.8751554750739244
0.004592667914161968 -0.8735528217460569
0.0045882016897721145 -0.871981061697839
0.00459123113821739 -0.8703004299891995
0.0045872357002572 -0.8688108303784463


## Sample and Noise Sensitivity

In [6]:
n = 3
d = 2**n
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

channel_target = CompactLindbladMap(d, 1, 1, 0.1)
model_ns1 = model_pipeline(channel_target, 
                           noise=1/np.sqrt(256), 
                           N_map=256, 
                           num_iter_spam = 2000,
                           num_iter_map = 2000)


model_ns2 = model_pipeline(channel_target, 
                           noise=1/np.sqrt(1024), 
                           N_map=1024, 
                           num_iter_spam = 2000,
                           num_iter_map = 2000)

model_ns3 = model_pipeline(channel_target, 
                           noise=1/np.sqrt(4096), 
                           N_map=4096, 
                           num_iter_spam = 2000,
                           num_iter_map = 2000)

saver([[channel_target],
       [model_ns1, model_ns2, model_ns3]], data_path("atypical_maps_robustness.model"))

  0%|          | 0/2000 [00:00<?, ?it/s]

0.002150885368496384


  0%|          | 0/2000 [00:00<?, ?it/s]

0.26794769945899877 -0.08766725008272647
0.05727617631030303 -0.32823873994945213
0.023443908262908056 -0.5578519809993621
0.016596680063794774 -0.6689932353624459
0.014566280268694824 -0.7200146732522673
0.013748146878470471 -0.74554355477519
0.013355446128449001 -0.7592869395858155
0.013146890185628162 -0.7671375099966475
0.013028770602231116 -0.7718333794992721
0.012958754711124698 -0.7747378601268832
0.012915776337280634 -0.7765735059087531
0.012888636937981457 -0.7777447955688811
0.01287108629413317 -0.7784894226712602
0.012859499829226614 -0.7789533336601742
0.012851708769692098 -0.779229680897788
0.012846381035251498 -0.7793798838413293
0.012842680124269158 -0.7794454517563968
0.012840070704480053 -0.7794549124603439
0.01283820455979948 -0.7794280417978202
0.012836851950982626 -0.779378590258415
0.012835859349478528 -0.7793160947892934


  0%|          | 0/2000 [00:00<?, ?it/s]

...

0.26183366025899574 -0.09546508173774945
0.05804923003313081 -0.3836870689402144
0.01456783014154421 -0.6782415743732303
0.008055271266233155 -0.7997469155404681
0.006442384859366886 -0.853576327015411
0.005854880202044879 -0.8814206197298448
0.0055759681995227 -0.89771676858529
0.0054225984661744695 -0.9082698126722878
0.005323488990180328 -0.9152490824309225
0.005266835251282968 -0.9199113661732512
0.005221052361611207 -0.9232531328518004
0.00519161745737921 -0.9258573499156537
0.0051731496263289645 -0.9277300584242826
0.005162893959857506 -0.9292405608296987
0.005146395630008088 -0.9301145195492124
0.005141348850941941 -0.9308924629328316
0.005133916955795202 -0.9311124721513038
0.005131786902931999 -0.931672102445257
0.005132760999965699 -0.9322880739258611
0.005122538857977105 -0.9322660155416707
0.005123109053077666 -0.9323865866372074


  0%|          | 0/2000 [00:00<?, ?it/s]

0.00021868318417711652


  0%|          | 0/2000 [00:00<?, ?it/s]

0.24485502378472027 -0.09180766146796303
0.0572850297124044 -0.37854715398471805
0.011308903999917297 -0.6923326312688468
0.004255240055662699 -0.8204710473790029
0.0026948115814801493 -0.8759911284844459
0.0021618700226277 -0.9057524723100916
0.0019293786221442144 -0.9237130516351342
0.0018038145377991103 -0.9359858183161975
0.0017307560544893206 -0.9445272729387377
0.0016835010144228837 -0.9508518717160213
0.0016517915097362208 -0.9557641896728687
0.0016310500874419777 -0.959356291152569
0.0016154433882695888 -0.9621214846120035
0.001604145290418941 -0.9644911330883145
0.001594863403115936 -0.9662791189616631
0.0015907851081693523 -0.9677623117918802
0.001584398841224131 -0.9690486095663288
0.0015819960065241042 -0.9698267403171676
0.0015799093556278935 -0.9707043303266133
0.0015767840657381043 -0.9712811836131505
0.001574551294343063 -0.9716889580011804
